In [20]:

import pandas as pd
import csv
from collections import Counter
from tqdm import tqdm


In [1]:
#displays given amount of messages from user in given file

def MinMaxFilter(readPath, minMessages, maxMessages):
    with open(readPath, mode='r', newline='', encoding='utf-8') as readFile:
        reader = csv.DictReader(readFile)
        userNames = []
        dropList = []
        keepList = []
        
        for row in reader: 
            if 'User' in row:
                userNames.append(row['User']) 

    counts = Counter(userNames)
    
    for name, count in counts.items():
        #print(f"{name}: {count}")
        if count >= minMessages and count <= maxMessages:
            keepList.append(name)
        else:
            dropList.append(name)

    data = pd.read_csv(readPath)
    data.drop(data[data['User'].isin(dropList)].index, inplace=True)
    return data, keepList

#stores the user names that we want to be included 
readPath = 'CleanText.csv'
minMessages = 100
maxMessages = 10000
#data, keepList = MinMaxFilter(readPath, minMessages, maxMessages)


In [2]:
#builds training message pairs with true false label that tells if it's from the same user or not
#will have a rewrite or different version done entirely in the future

def PairData(data, keepList, writePath, messagesPerColumn):
    data = data.sort_values(by=['User']).reset_index(drop=True)
    #data.sort_values(by=['User'])
    entries = []
    row = 0
    #builds one true case and one false case requiring 3 message chunks from same user and 1 from some one else.
    messagesFromUser = messagesPerColumn * 3
    messagesInTotal = messagesPerColumn + messagesFromUser
    
    for user in tqdm(keepList, desc="Processing users"):

        #from the same user
        messages = []
        messageCount = 0
        
        while messageCount < messagesFromUser:
            if data.iloc[row]['User'] == user:
                messages.append(str(data.iloc[row]['Message']))
                messageCount += 1
            if row + 1 >= len(data):
                row = 0
            row += 1

        messages1 = messages[:int(messagesInTotal / 4)]
        messages2 = messages[int(messagesInTotal / 4):int(messagesInTotal * 2 / 4)]
        fromSameUser = True
        
        entries.append([' '.join(messages1), ' '.join(messages2), fromSameUser])

        
        #from different user
        while messageCount < messagesInTotal:
            if data.iloc[row]['User'] != user:
                messages.append(str(data.iloc[row]['Message']))
                messageCount += 1
            if row + 1 >= len(data):
                row = 0
            row += 1

        messages1 = messages[int(messagesInTotal * 2 / 4):int(messagesInTotal * 3 / 4)]
        messages2 = messages[int(messagesInTotal * 3 / 4):]
        fromSameUser = False
        
        entries.append([' '.join(messages1), ' '.join(messages2), fromSameUser])
        
    with open(writePath, 'w', newline='', encoding='utf-8') as writeFile:
        writer = csv.writer(writeFile)
        #Header
        writer.writerow(['Message1', 'Message2', 'FromSameUser'])
        writer.writerows(entries)

#print(len(data))
messagesPerColumn = 30
writePath = 'TrainingData1.csv'
#PairData(data, keepList, writePath, messagesPerColumn)